In [1]:
import warnings
warnings.filterwarnings("ignore")
from joblib import Parallel, delayed
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from itertools import combinations

from IWLS import IWLS, adaptive_IWLS
from first_order import first_order
from margin import margin

In [2]:
# general parameters
n, k = 50, 5
job_n = 50

seed = 23
cov = 2.0

target = "test_loss"
np.random.seed(seed)

# generate data
mean_n = np.array([-1, 0])
mean_p = np.array([1, 0])
covariance = np.eye(2) * cov
x_n = np.random.multivariate_normal(mean_n, covariance, int(n/2))
x_p = np.random.multivariate_normal(mean_p, covariance, int(n/2))

y_n = np.zeros(int(n/2)) # 0 labels
y_p = np.ones(int(n/2))  # 1 labels

X_train = np.vstack((x_n, x_p))
y_train = np.hstack((y_n, y_p))

# Choose mean_n or mean_p w.p. 1/2
if np.random.rand() < 0.5:
	x_test = np.random.multivariate_normal(mean_n, covariance)
	y_test = 0
else:
	x_test = np.random.multivariate_normal(mean_p, covariance)
	y_test = 1

In [3]:
IWLS_best = IWLS(X_train, y_train, x_test, y_test, target=target)
adaptive_IWLS_best_k = adaptive_IWLS(X_train, y_train, x_test, y_test, k=k, target=target)
ind_n, ind_p = margin(X_train, y_train)
FO_best = first_order(X_train, y_train, x_test, y_test, target=target)

print(IWLS_best)

[48 34 42 15 27 16 47 49 39 18 43  9  4 28 37 30  7 41 44 31 38  1 26 45
 33 25 35 17  5 24 10 13 46  3 40 32  8 23 29 20 11 22 36 21  6  0 19 14
 12  2]
